In [1]:
from influxdb import InfluxDBClient
import pandas as pd
from sklearn.ensemble import IsolationForest
import joblib
import pickle
# Connect to the InfluxDB server
client = InfluxDBClient(host='192.168.0.75', port=8086, username='admin', password='Stzpwk35i@gF2P', database='iotedge_db')

In [2]:
# Define the queries
query = 'SELECT max("temperature") as temperature, max("humidity") as humidity FROM "dht22" WHERE time > now() - 3d GROUP BY time(10s)'

In [3]:
result = client.query(query)
df = pd.DataFrame(result.get_points())

# Convert the timestamp to a datetime object and set it as the index
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df.dropna()

In [5]:
model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
model.fit(df)

/home/lars-linux/coding/iot_edge/.venv/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


IsolationForest(contamination=0.01, random_state=42)

In [6]:
with open('isolation_forest_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [7]:
anomaly_scores = model.decision_function(df)

In [8]:
df['scores'] = anomaly_scores

In [12]:
df[df['scores']>0.2]

,temperature,humidity,scores
time,,,
2023-04-10 12:12:10+00:00,20.9,41.8,0.211040
2023-04-10 12:12:30+00:00,20.9,41.7,0.214743
2023-04-10 12:12:40+00:00,20.9,41.7,0.214743
2023-04-10 12:12:50+00:00,20.9,41.7,0.214743
2023-04-10 12:13:40+00:00,20.9,41.8,0.211040
...,...,...,...
2023-04-13 11:51:40+00:00,21.1,41.3,0.203292
2023-04-13 11:51:50+00:00,21.1,43.4,0.202345
2023-04-13 11:53:40+00:00,21.1,43.4,0.202345
